In [3]:
#https://naomi-fridman.medium.com/install-pyspark-to-run-on-jupyter-notebook-on-windows-4ec2009de21f
import findspark

In [4]:
findspark.init()
findspark.find()
import pyspark

#### Examples to test working of spark

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
nums = sc.parallelize([1,2,3,4])
nums.map(lambda x: x*x).collect()

[1, 4, 9, 16]

In [3]:
from pyspark import SparkContext
import numpy as np
sc=SparkContext(master="local[4]")
words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'computers'

In [3]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext()
sqlContext = SQLContext(sc)
list_p = [('John',19),('Smith',29),('Adam',35),('Henry',50)]
rdd = sc.parallelize(list_p)
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = sqlContext.createDataFrame(ppl)
DF_ppl.printSchema()
DF_ppl.show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-----+---+
| name|age|
+-----+---+
| John| 19|
|Smith| 29|
| Adam| 35|
|Henry| 50|
+-----+---+



In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession \
        .builder \
        .appName('First spark session') \
        .getOrCreate()

In [7]:
spark.sparkContext.getConf().getAll()

[('spark.driver.port', '54891'),
 ('spark.driver.host', '192.168.1.8'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'First spark session'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1603841262293'),
 ('spark.ui.showConsoleProgress', 'true')]

In [62]:
usr_log = spark.read.json(r'C:\Users\Shrikanth Singh\Desktop\Conrad Notebooks\Py-Spark Practice\Sparkify-Project-master\sparkify_log_small.json')

In [63]:
usr_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [64]:
usr_log.show(n=2)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [65]:
out_path = 'sparkify_log_small.csv'
#usr_log.write.save(out_path, format='csv', header=True)

In [66]:
usr_log_csv = spark.read.csv(out_path)

In [67]:
usr_log.describe('sessionId').show()

+-------+------------------+
|summary|         sessionId|
+-------+------------------+
|  count|             10000|
|   mean|         4436.7511|
| stddev|2043.1281541827561|
|    min|                 9|
|    max|              7144|
+-------+------------------+



In [68]:
usr_log.select(['page']).dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [69]:
usr_log.select(['userId', 'status', 'artist', 'song', 'ts']).where(usr_log.userId=='1046').show()

+------+------+------------------+--------------------+-------------+
|userId|status|            artist|                song|           ts|
+------+------+------------------+--------------------+-------------+
|  1046|   200|     Showaddywaddy|Christmas Tears W...|1513720872284|
|  1046|   200|     Darius Rucker|  Be Wary Of A Woman|1513721104284|
|  1046|   200|      Public Enemy|   Public Enemy No.1|1513721310284|
|  1046|   200|        Jag Panzer|Reign Of The Tyrants|1513721591284|
|  1046|   200|           Boyzone|      Father And Son|1513721808284|
|  1046|   200|  Hollywood Undead|               No. 5|1513721976284|
|  1046|   200|   Jimmy Eat World|           Seventeen|1513722161284|
|  1046|   200|              null|                null|1513837636284|
|  1046|   200|             Wilco|          War on war|1513837660284|
|  1046|   200|Fountains Of Wayne|   Killermont Street|1513837887284|
|  1046|   200|        Miike Snow|        Black & Blue|1513838080284|
|  1046|   307|     

In [70]:
from pyspark.sql.functions import udf
import datetime
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).hour)

In [71]:
usr_log = usr_log.withColumn('hour', get_hour(usr_log.ts))

In [72]:
usr_log.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='23')

In [73]:
songs_in_hour = usr_log.where(usr_log.page=='NextSong').groupBy(usr_log.hour).count().orderBy(usr_log.hour.cast('float'))

In [74]:
usr_log_valid = usr_log.dropna(how='any', subset=['sessionId', 'userId']) 

In [75]:
usr_log_valid.count()

10000

In [76]:
usr_log_valid = usr_log_valid.where(usr_log_valid.userId != '')
usr_log_valid.count()

9664

In [77]:
usr_log_valid.where(usr_log_valid.page == 'Submit Downgrade').show()

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|            page| registration|sessionId|song|status|           ts|           userAgent|userId|hour|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+
|  null|Logged In|    Kelly|     F|           24|  Newton|  null| paid|Houston-The Woodl...|   PUT|Submit Downgrade|1513283366284|     5931|null|   307|1513768454284|Mozilla/5.0 (Wind...|  1138|  12|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+


In [25]:
usr_log.select(['userId', 'firstName', 'page', 'level']).where(usr_log.userId=='1138').collect()

[Row(userId='1138', firstName='Kelly', page='Home', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid'),
 Row(userI

In [78]:
from pyspark.sql.types import *
flag_downgrade_event = udf(lambda x: 1 if x == 'Submit Downgrade' else 0, IntegerType())
usr_log_valid = usr_log_valid.withColumn('flag_event', flag_downgrade_event(usr_log_valid.page))

In [79]:
usr_log_valid.show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+----+----------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|hour|flag_event|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+----+----------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|  23|         0|
|          Lily Allen|Logged In|Elizabeth|     F|   

In [80]:
from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
import pyspark.sql.functions as F
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [93]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

In [94]:
usr_log_valid = usr_log_valid.withColumn("list_view", F.collect_list(F.col("flag_event")).over(windowval))
usr_log_valid = usr_log_valid.withColumn("phase", Fsum("flag_event").over(windowval))

In [95]:
usr_log_valid.select(['page','level', 'list_view', 'phase']).where(usr_log_valid.userId == '1138').show()

+----------------+-----+--------------------+-----+
|            page|level|           list_view|phase|
+----------------+-----+--------------------+-----+
|        NextSong| free|                 [0]|    0|
|            Home| free|              [0, 0]|    0|
|        NextSong| free|           [0, 0, 0]|    0|
|            Home| free|        [0, 0, 0, 0]|    0|
|Submit Downgrade| paid|     [0, 0, 0, 0, 1]|    1|
|       Downgrade| paid|  [0, 0, 0, 0, 1, 0]|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        NextSong| paid|[0, 0, 0, 0, 1, 0...|    1|
|        Nex